In [1]:
import requests
import pandas as pd
from datetime import datetime
import os

API_URL = "https://gateway.api.bot.or.th/Stat-ExchangeRate/v2/MONTHLY_AVG_EXG_RATE/"
API_KEY = "eyJvcmciOiI2NzM1NzgwZWM4YzFlYjAwMDEyYTM3NzEiLCJpZCI6Ijg5YWNlOTc2NzllNTQxOGQ4MjEyNTU2YzgyMzUxNzVlIiwiaCI6Im11cm11cjEyOCJ9"
RAW_FILE = "data/raw/exchange_rate.csv"

def fetch_exchange_rate():
    params = {
        "start_period": "2025-10",
        "end_period": "2025-11",
        "currency": "USD"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": API_KEY   # ✅ ใช้ Authorization แทน X-IBM-Client-Id
    }

    resp = requests.get(API_URL, headers=headers, params=params)
    resp.raise_for_status()
    data = resp.json()

    details = data["result"]["data"]["data_detail"]
    df = pd.DataFrame(details)
    df["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return df

def save_to_csv(df):
    os.makedirs(os.path.dirname(RAW_FILE), exist_ok=True)
    if os.path.exists(RAW_FILE):
        old_df = pd.read_csv(RAW_FILE)
        df_all = pd.concat([old_df, df], ignore_index=True).drop_duplicates(subset=["period", "currency_id"], keep="last")
    else:
        df_all = df
    df_all.to_csv(RAW_FILE, index=False, encoding="utf-8-sig")
    print(f"✅ Saved data to {RAW_FILE}")

def main():
    df_new = fetch_exchange_rate()
    save_to_csv(df_new)

if __name__ == "__main__":
    main()


✅ Saved data to data/raw/exchange_rate.csv
